In [1]:
import requests
import csv
import re
from bs4 import BeautifulSoup
import pandas as pd
import datetime
from datetime import date

In [144]:
team_urls = pd.read_csv('Spiele_Links.csv')
idx = 8
print(team_urls['team'][idx])
url = team_urls['url'][idx]
response = requests.get(url)

D2


In [145]:
#BeautifulSoup HTML-Dokument aus dem Quelltext parsen
soup= BeautifulSoup(response.text, 'html.parser')

In [146]:
#Tabelle aus dem HTML-Dokument extrahieren
table = soup.find('div', {"id":'id-team-matchplan-table'})
#print(table)

In [147]:
team=team_urls['team'][idx]
#Datum und Liga auslesen (befindet sich in jeder 3. Zeile ab 2. Zeile)
for data in table.find_all('tbody'):
    rows = data.find_all('tr')
    dates = list()
    times = list()
    teams = list()
    kw = list()
    for i in range(1,len(rows),3):
        #entfernt Sonderzeichen wie Zeilenumbruch (n) und Tab (t)
        date = rows[i].find('td', class_='column-date') #rows[i].text
        cleanString = re.sub(r'\ \|',',', date.text)
        cleanString = re.sub(r' ', '', cleanString)
        #removedSpace = re.sub(r'\t', '', removedSpace)
        splitted= cleanString.split(',')
        date_split = splitted[1].split('.')
        datum = datetime.date(int(date_split[2])+2000, int(date_split[1]), int(date_split[0]))
        kw.append(datum.isocalendar()[1])
        dates.append(splitted[1])
        times.append(splitted[2].encode('utf-8').decode('utf-8'))
        teams.append(team)
        
df = pd.DataFrame(list(zip(dates,times,teams)),columns=['Datum','Zeit','Team'])
df.insert(len(df.columns), 'KW', kw)

In [148]:
#Club und Logo auslesen (befindet sich in jeder 3. Zeile ab 3. Zeile)
for data in table.find_all('tbody'):
    rows = data.find_all('tr')
    club1 = list()
    club2 = list()
    for i in range(2,len(rows),3):
        #entfernt Sonderzeichen wie Zeilenumbruch (n) und Tab (t)
        #print(rows[i].text)
        removedSpace = re.sub(r'\n', '', rows[i].text)
        removedSpace = re.sub(r'\t', '', removedSpace)
        removedSpace = re.sub(r':', ';', removedSpace)
        splitted = removedSpace.split(';')
        club1.append(splitted[0])
        club2.append(splitted[1][0:-1])

df.insert(len(df.columns), "Heim", club1)
df.insert(len(df.columns), "Gast", club2)

In [149]:
#Datum und Liga auslesen (befindet sich in jeder 3. Zeile ab 2. Zeile)
for data in table.find_all('tbody'):
    rows = data.find_all('tr')
    logos_home = list()
    logos_guest= list()
    for i in range(2,len(rows),3):
        #entfernt Sonderzeichen wie Zeilenumbruch (n) und Tab (t)
        logo = rows[i].find_all('div', class_="club-logo") #rows[i].text
        
        logo_link = re.findall(r'//(\S+)\">', str(logo))
        logos_home.append(logo_link[0])
        logos_guest.append(logo_link[1])
        
#found = re.search('AAA(.+?)ZZZ', string).group(1)

df.insert(len(df.columns), 'Logo Heim', logos_home)
df.insert(len(df.columns), 'Logo Gast', logos_guest)

In [150]:
df

,Datum,Zeit,Team,KW,Heim,Gast,Logo Heim,Logo Gast
0,22.04.23,09:30,D2,16,SGM Ilsfeld-Abstatt-Beilstein ABI II,TSG Heilbronn II,www.fussball.de/export.media/-/action/getLogo/...,www.fussball.de/export.media/-/action/getLogo/...
1,24.04.23,18:15,D2,17,SGM Nordhausen Unteres Zabergäu II,SGM Ilsfeld-Abstatt-Beilstein ABI II,www.fussball.de/export.media/-/action/getLogo/...,www.fussball.de/export.media/-/action/getLogo/...
2,29.04.23,11:10,D2,17,SGM Cleebronn/​Botenheim/​Eibensbach/​Stockheim,SGM Ilsfeld-Abstatt-Beilstein ABI II,www.fussball.de/export.media/-/action/getLogo/...,www.fussball.de/export.media/-/action/getLogo/...
3,06.05.23,09:30,D2,18,SGM Ilsfeld-Abstatt-Beilstein ABI II,SGM Dahenfeld/​Oedheim II,www.fussball.de/export.media/-/action/getLogo/...,www.fussball.de/export.media/-/action/getLogo/...
4,10.05.23,18:00,D2,19,SGM TSV Hardthausen KoBra I,SGM Ilsfeld-Abstatt-Beilstein ABI II,www.fussball.de/export.media/-/action/getLogo/...,www.fussball.de/export.media/-/action/getLogo/...
5,20.05.23,09:30,D2,20,SGM Ilsfeld-Abstatt-Beilstein ABI II,TV Flein II,www.fussball.de/export.media/-/action/getLogo/...,www.fussball.de/export.media/-/action/getLogo/...
6,17.06.23,11:10,D2,24,SC Amorbach,SGM Ilsfeld-Abstatt-Beilstein ABI II,www.fussball.de/export.media/-/action/getLogo/...,www.fussball.de/export.media/-/action/getLogo/...


In [151]:
links = table.find_all('a')
homeLinks = list();
guestLinks = list();
spielLinks = list();
for i in range(2, len(links),6):
    homeLinks.append(links[i].get('href'))
    guestLinks.append(links[i+1].get('href'))
    spielLinks.append(links[i+2].get('href'))
df.insert(len(df.columns), "home_link", homeLinks)
df.insert(len(df.columns), 'guest_link', guestLinks)
df.insert(len(df.columns), 'Spiel', spielLinks)
df

,Datum,Zeit,Team,KW,Heim,Gast,Logo Heim,Logo Gast,home_link,guest_link,Spiel
0,22.04.23,09:30,D2,16,SGM Ilsfeld-Abstatt-Beilstein ABI II,TSG Heilbronn II,www.fussball.de/export.media/-/action/getLogo/...,www.fussball.de/export.media/-/action/getLogo/...,https://www.fussball.de/mannschaft/sgm-ilsfeld...,https://www.fussball.de/mannschaft/tsg-heilbro...,https://www.fussball.de/spiel/sgm-ilsfeld-abst...
1,24.04.23,18:15,D2,17,SGM Nordhausen Unteres Zabergäu II,SGM Ilsfeld-Abstatt-Beilstein ABI II,www.fussball.de/export.media/-/action/getLogo/...,www.fussball.de/export.media/-/action/getLogo/...,https://www.fussball.de/mannschaft/sgm-nordhau...,https://www.fussball.de/mannschaft/sgm-ilsfeld...,https://www.fussball.de/spiel/sgm-nordhausen-u...
2,29.04.23,11:10,D2,17,SGM Cleebronn/​Botenheim/​Eibensbach/​Stockheim,SGM Ilsfeld-Abstatt-Beilstein ABI II,www.fussball.de/export.media/-/action/getLogo/...,www.fussball.de/export.media/-/action/getLogo/...,https://www.fussball.de/mannschaft/sgm-cleebro...,https://www.fussball.de/mannschaft/sgm-ilsfeld...,https://www.fussball.de/spiel/sgm-cleebronn-bo...
3,06.05.23,09:30,D2,18,SGM Ilsfeld-Abstatt-Beilstein ABI II,SGM Dahenfeld/​Oedheim II,www.fussball.de/export.media/-/action/getLogo/...,www.fussball.de/export.media/-/action/getLogo/...,https://www.fussball.de/mannschaft/sgm-ilsfeld...,https://www.fussball.de/mannschaft/sgm-dahenfe...,https://www.fussball.de/spiel/sgm-ilsfeld-abst...
4,10.05.23,18:00,D2,19,SGM TSV Hardthausen KoBra I,SGM Ilsfeld-Abstatt-Beilstein ABI II,www.fussball.de/export.media/-/action/getLogo/...,www.fussball.de/export.media/-/action/getLogo/...,https://www.fussball.de/mannschaft/sgm-tsv-har...,https://www.fussball.de/mannschaft/sgm-ilsfeld...,https://www.fussball.de/spiel/sgm-tsv-hardthau...
5,20.05.23,09:30,D2,20,SGM Ilsfeld-Abstatt-Beilstein ABI II,TV Flein II,www.fussball.de/export.media/-/action/getLogo/...,www.fussball.de/export.media/-/action/getLogo/...,https://www.fussball.de/mannschaft/sgm-ilsfeld...,https://www.fussball.de/mannschaft/tv-flein-ii...,https://www.fussball.de/spiel/sgm-ilsfeld-abst...
6,17.06.23,11:10,D2,24,SC Amorbach,SGM Ilsfeld-Abstatt-Beilstein ABI II,www.fussball.de/export.media/-/action/getLogo/...,www.fussball.de/export.media/-/action/getLogo/...,https://www.fussball.de/mannschaft/sc-amorbach...,https://www.fussball.de/mannschaft/sgm-ilsfeld...,https://www.fussball.de/spiel/sc-amorbach-sgm-...


In [152]:
#merge columns to table
if (idx==0):
    df.to_csv('out.csv', index=False, encoding="utf-8")
else:
    df.to_csv('out.csv', mode='a', index=False, header=False, encoding="utf-8")

